In [1]:
import pandas as pd
import mysql.connector as connector
from pypika import Table, Query

In [2]:
db_connector = connector.connect(
                    host="localhost",
                    user="root",
                    passwd="",
                    database="nest-bot-dev"
                )

In [3]:
df = pd.read_excel('db_alker.xlsx')
df

,NO,NIK,NAMA,Serial Number,Jenis Barang,Merk Barang,Nama Product,STATUS
0,1,20961102,ABDUL ROKHMAN,1503201128,OPTICAL POWER METER,B&D TECHNOLOGY,OPTICAL POWER METER B&D TECHNOLOGY BD-503,BAIK
1,2,18950823,RACHMAT HENDRA SAPUTRA,1503201133,OPTICAL POWER METER,B&D TECHNOLOGY,OPTICAL POWER METER B&D TECHNOLOGY BD-503,BAIK
2,3,18980377,RIZALDI ILHAM PRATAMA,1503201153,OPTICAL POWER METER,B&D TECHNOLOGY,OPTICAL POWER METER B&D TECHNOLOGY BD-503,BAIK
3,4,20961126,NINKA DEARANI,1503201156,OPTICAL POWER METER,B&D TECHNOLOGY,OPTICAL POWER METER B&D TECHNOLOGY BD-503,BAIK
4,5,91160082,AJIE RAMADHIANTO,1503301375,OPTICAL POWER METER,B&D TECHNOLOGY,OPTICAL POWER METER B&D TECHNOLOGY BD-503,BAIK
5,6,18910017,CHARIS,1503301376,OPTICAL POWER METER,B&D TECHNOLOGY,OPTICAL POWER METER B&D TECHNOLOGY BD-503,BAIK
6,7,18880071,KHARIS NAGARA,1503301492,OPTICAL POWER METER,B&D TECHNOLOGY,OPTICAL POWER METER B&D TECHNOLOGY BD-503,HILANG
7,8,18930234,AKHMAD URIFAN,1503301497,OPTICAL POWER METER,B&D TECHNOLOGY,OPTICAL POWER METER B&D TECHNOLOGY BD-503,BAIK
8,9,91160066,ALDIAN AKBAR NAUFAL,GAF0390577,OPTICAL POWER METER,B&D TECHNOLOGY,OPTICAL POWER METER B&D TECHNOLOGY BD-503,HILANG
9,10,18950014,MUKHAMMAD MUDASIR SUTRISNO,GSF0390594,OPTICAL POWER METER,B&D TECHNOLOGY,OPTICAL POWER METER B&D TECHNOLOGY BD-503,HILANG


In [14]:
len(df)

25

In [6]:
list_nik = df['NIK'].to_list()
list_nama = df['NAMA'].to_list()
assert len(list_nik) == len(list_nama)

In [13]:
for i, v in enumerate(list_nik):
    cursor = db_connector.cursor(buffered=True)
    tabl = Table('employees')
    q = Query.into(tabl).columns(tabl.full_name,tabl.nik).insert(list_nama[i],v)
    cursor.execute(q.get_sql(quote_char=None))
    db_connector.commit()

In [20]:
merk_barang = list(set(df['Merk Barang']))

In [23]:
dict_merk = {}
start_idx = 1
for merk in merk_barang:
    dict_merk[merk] = start_idx
    start_idx += 1

In [24]:
dict_merk

{'B&D TECHNOLOGY': 1, 'JOINWIT FROM SANFIX': 2}

In [17]:
dict_nik = {}
start_idx = 1
for nik in list_nik:
    dict_nik[nik] = start_idx
    start_idx += 1

In [29]:
for idx, row in df.iterrows():
    nik = int(row['NIK'])
    serial = str(row['Serial Number']).upper().strip()
    jb = 1
    merk = dict_merk[str(row['Merk Barang'])]
    pd = str(row['Nama Product']).upper().strip()
    status = str(row['STATUS']).lower().strip()
    cursor = db_connector.cursor(buffered=True)
    table_products = Table('products')
    q1 = Query.into(table_products).columns(table_products.serial_number, table_products.product_name, table_products.merk_id, table_products.jenis_id)\
    .insert(serial, pd, merk, jb)
    cursor.execute(q1.get_sql(quote_char=None))
    db_connector.commit()
    emp_id = idx + 1
    tab_position = Table('position_products')
    q2 = Query.into(tab_position).columns(tab_position.new_update, tab_position.employee_id, tab_position.product_id, tab_position.approve_state, tab_position.product_state)\
    .insert(1, emp_id, (idx+1), 'approved', status)
    cursor.execute(q2.get_sql(quote_char=None))
    db_connector.commit()

IntegrityError: 1452 (23000): Cannot add or update a child row: a foreign key constraint fails (`nest-bot-dev`.`position_products`, CONSTRAINT `position_products_employee_id_foreign` FOREIGN KEY (`employee_id`) REFERENCES `employees` (`id`) ON DELETE CASCADE ON UPDATE CASCADE)